<a id='top'></a>

# CSCI 3202, Spring 2020
# Assignment 3
# Due:  Friday 6 March 2020 by 11:59 PM

<br>

### Your name: Jason Nguyen

<br>




**Some things that might be useful**


In [1]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time

<br>

---

## Problem 1 (35 points):  Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>

<a id='p2a'></a>

### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **2b** and **2d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = moves

class TicTacToe:
    def __init__(self, nrow=3, ncol=3, nwin=3):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''        
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        return state.utility if player=='X' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    return self.state.utility 

<br>


### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
def random_player(game):
    '''A player that chooses a legal move at random.'''
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]


<br>


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **2b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [4]:
# Your code here.
niter = 10000
wins = 0
draws = 0
losses = 0
for k in range(niter):
    ttt = TicTacToe(3,3,3)
    out = ttt.play_game(random_player, random_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage = {}'.format(wins/niter))
print('First-player losing percentage = {}'.format(losses/niter))
print('First-player draws percentage = {}'.format(draws/niter))

First-player winning percentage = 0.5877
First-player losing percentage = 0.2858
First-player draws percentage = 0.1265


For a 4x4 board:
- First-player winning percentage = 0.3144
- First-player losing percentage = 0.2591
- First-player draws percentage = 0.4265

For a 3x3 board:
- First-player winning percentage = 0.5907
- First-player losing percentage = 0.2828
- First-player draws percentage = 0.1265

From the data above, each simulation was run with 10,000 iterations. You will notice that the chances that the second player wins in a 4x4 slightly increases while the chances that the first player wins decreases significantly. The amount of draws also rises significantly going from 12% of the time to a staggering 42%. Honestly my predictions would have been that the number of draws would have decreased but possibly since there is more turns that the players have to influence the end of the game, the draws increase.


<br>


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [10]:
# Your code here.
def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    '''search game approach to find best action, using alpha-beta pruning:
    alpha = best (highest) move found so far for Max
    beta  = best (lowest) move found so far for Min'''

    player = game.state.to_move
    
    
    possible_moves = game.state.moves

    # Functions used by alphabeta
    # took a lot of the structure of the code from page 20 of lecture 12
    def max_value(state, alpha, beta):
        
        #if the game is over, return the win or lose
        if game.game_over(state):
            return state.utility
        
        #initialize the v value
        value = -9999999
        
        for action in state.moves:
            
            new_val = min_value(game.result(action, state), alpha, beta)
            if value < new_val:
                state.minimax = action
                value = new_val
            
            if value >=  beta: return value
            alpha = max(value, alpha)
        
        return value

    def min_value(state, alpha, beta):
        
         #if the game is over, return the win or lose
        if game.game_over(state):
            return state.utility
        
        #initialize the v value, positive infinity
        value = 9999999
        
        for action in state.moves:
            
            new_val = max_value(game.result(action, state), alpha, beta)
            if value > new_val:
                state.minimax = action
                value = new_val
            
#             value = min(value, max_value(game.result(action, state), alpha, beta))
            
            if value <=  alpha: return value 
            beta = min(value, beta)
            
        return value
        
        
    # Body of alphabeta_cutoff_search:
    
    #alpha beta pruning
    alpha = -9999999999
    beta = 9999999999
    
    #use max value if the player goes first. Choose the minval if player is second
    if game.state.to_move == 'X':
        max_value(game.state, alpha, beta)
    else:
        min_value(game.state, alpha, beta)
     
    return game.state.minimax




ttt = TicTacToe(3,3,3)
out = ttt.play_game(alphabeta_player, random_player)
out


1

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **only 10 games for each test**, and track the number of wins, draws and losses.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. 

In [23]:
# Your code here.
niter = 10
wins = 0
draws = 0
losses = 0
for k in range(niter):
    ttt = TicTacToe(3,3,3)
    out = ttt.play_game(alphabeta_player, alphabeta_player)
    if out==1:
        wins += 1
    elif out==-1:
        losses += 1
    else:
        draws += 1
        
print('First-player winning percentage = {}'.format(wins/niter))
print('First-player losing percentage = {}'.format(losses/niter))
print('First-player draws percentage = {}'.format(draws/niter))

First-player winning percentage = 0.0
First-player losing percentage = 0.0
First-player draws percentage = 1.0


Alpha Beta First:
- First-player winning percentage = 1.0
- First-player losing percentage = 0.0
- First-player draws percentage = 0.0

Alpha Beta Second:
- First-player winning percentage = 0.0
- First-player losing percentage = 0.8
- First-player draws percentage = 0.2

Alpha Beta vs. Alpha Beta:
- First-player winning percentage = 0.0
- First-player losing percentage = 0.0
- First-player draws percentage = 1.0





<br>


### (1e) What has pruning ever done for us?

Calculate the number of states expanded by the minimax algorithm, with and without pruning, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [65]:
#WITH ALPHA BETA PRUNING
def alphabeta_player_recurr(game):
    return alphabeta_search_recurr(game)

def alphabeta_search_recurr(game):

    player = game.state.to_move

    possible_moves = game.state.moves

    def max_value(state, alpha, beta):
        
        global counter
        counter += 1
        
        #if the game is over, return the win or lose
        if game.game_over(state):
            return state.utility
        
        #initialize the v value
        value = -9999999
        
        for action in state.moves:
            
            new_val = min_value(game.result(action, state), alpha, beta)
            if value < new_val:
                state.minimax = action
                value = new_val
            if value >=  beta: return value
            alpha = max(value, alpha)
        
        return value

    def min_value(state, alpha, beta):
        
        global counter
        counter += 1

         #if the game is over, return the win or lose
        if game.game_over(state):
            return state.utility
        
        #initialize the v value, positive infinity
        value = 9999999
        
        for action in state.moves:
            
            new_val = max_value(game.result(action, state), alpha, beta)
            if value > new_val:
                state.minimax = action
                value = new_val

            if value <=  alpha: return value 
            beta = min(value, beta)
            
        return value
        
    alpha = -9999999999
    beta = 9999999999
    
    
    if game.state.to_move == 'X':
        max_value(game.state, alpha, beta)
    else:
        min_value(game.state, alpha, beta)
     
    return game.state.minimax

counter = 0
ttt = TicTacToe(3,3,3)
#out = ttt.play_game(alphabeta_player_recurr, random_player)
alphabeta_player_recurr(ttt)
counter



18297

In [41]:
#WITH *NO* ALPHA BETA PRUNING
def alphabeta_player_recurr_no_prune(game):
    return alphabeta_search_recurr_no_prune(game)

def alphabeta_search_recurr_no_prune(game):

    player = game.state.to_move

    possible_moves = game.state.moves

    def max_value(state, alpha, beta):
        
        global counter
        counter += 1
        
        #if the game is over, return the win or lose
        if game.game_over(state):
            return state.utility
        
        #initialize the v value
        value = -9999999
        
        for action in state.moves:
            
            new_val = min_value(game.result(action, state), alpha, beta)
            if value < new_val:
                state.minimax = action
                value = new_val
#             if value >=  beta: return value
#             alpha = max(value, alpha)
        
        return value

    def min_value(state, alpha, beta):
        
        global counter
        counter += 1

         #if the game is over, return the win or lose
        if game.game_over(state):
            return state.utility
        
        #initialize the v value, positive infinity
        value = 9999999
        
        for action in state.moves:
            
            new_val = max_value(game.result(action, state), alpha, beta)
            if value > new_val:
                state.minimax = action
                value = new_val

#             if value <=  alpha: return value 
#             beta = min(value, beta)
            
        return value
        
    alpha = -9999999999
    beta = 9999999999
    
    
    if game.state.to_move == 'X':
        max_value(game.state, alpha, beta)
    else:
        min_value(game.state, alpha, beta)
     
    return game.state.minimax

counter = 0
ttt = TicTacToe(3,3,3)
#out = ttt.play_game(alphabeta_player_recurr_no_prune, random_player)
alphabeta_player_recurr_no_prune(ttt)
counter

549946

549946, 18297

Pruning helps us a lot!

I found that when we do prune we recurr about 18297 times and when we do not prune we will get around 549946 recurrsions. That means with pruning we run the function about 30 times faster!

<br>

---

## Problem 2 (30 points):  Uncertainty

### (2a) 

Suppose you are deciding when to arrive at a party. There is some optimal time to arrive when the loss you feel, as measured by _awkwardness_, is minimized at 0. That is, at some particular time, it is not awkward at all to show up to the party. The awkwardness (loss) increases as you arrive too early or too late relative to this optimal time. What is a suitable loss function, $L(d, x)$, to model this situation? Include definitions for $d$ and $x$, consistent with the examples from this class. Use this loss function this weekend when you go out.

$L(d,x)$ is loss when we make decision $d$ in state $x$

$d$ is the decision you make

$x$ is the state variable

A suitable loss function would be using Bayes decision that minimizes our expected loss or the minimized awkwardness. For my example, we can think of the following L(d,x) function: 

$L(d, x) = C(d-x)^2$

### (2b)

Suppose we have a situation where loss is given by the function $L(d, x) = 2(d-x)^2$. Set up, simplify, and evaluate integral(s) for the expected loss, $E_x[L(d, x)]$, where your prior beliefs regarding $x$ follow the distribution $f(x)$ given below. You may assume $f(x)=0$ for values of $x$ outside of the interval $[0, 3]$.

f(x) =  \begin{cases} 
      1/2 & 0 \leq x <1 \\
      3/8 & 1 \leq x <2 \\
      1/8 & 2 \leq x \leq 3 
   \end{cases}


 $E_x = 1/2\int_{0}^{1} 2(d-x)^2 + 3/8\int_{1}^{2} 2(d-x)^2 + 1/8\int_{2}^{3} 2(d-x)^2$
 
 The first integral is: $3d^2 - 3d + 1/3$
 
 The second integral is: $\frac{6d^2-18d+14}{8}$
 
 The third integral is: $\frac{6d^2 - 30d+38}{3*8}$
 
 $3d^2 - 3d + 1/3 + \frac{1}{24}(6d^2 - 30d+38) + \frac{1}{8}(6d^2-18d+14)$

### (2c) 

Suppose our expected loss is represented by the function $E_x[(L(d,x)]=(2-d)^2+2$, and our prior beliefs regarding $x$ are given by the distribution $f(x)$ from part b.

- Calculate Bayes' Decision, $d_{Bayes}$.

- Calculate the Expected Value of Including Uncertainty, EVIU. Suppose that if we ignore uncertainty, our best guess for what decision to make is the median of $x$ (under our prior $f(x)$).

$d_{bayes}$ is equal to the minimum of the $E_x[(L(d,x)]$ which will equal 2. 
- $d_{bayes}$ = $(2-2)^2+2 = 2$.

$d_{iu}$ will equal 2 because it is 1/2 for 0 to 1 and $\frac{3}{8}+\frac{1}{8}$ for 1 to 2 and you multiply it by 2 because of the 2 in L(d,x). Therefore, $𝐿(d_{iu},x)$ - $𝐿(d_{bayes},x)$ = 1.


$(2-d_{iu})^2+2 - (2-d_{bayes})^2+2 = (2-1)^2+2 - (2-2)^2+2 = 1^2+2 - 0+2 = 3 - 2 = 1$ 


<br>

## Problem 3 (35 points): Maximizing some objective function with a Genetic Algorithm

Suppose we are trying to figure out a cookie recipe, but can't quite remember how much of each ingredient we need.

So we want to maximize the following objective function corresponding to how close we are to this recipe:

* 3/4 cup granulated sugar (36 tsp)
* 3/4 cup packed brown sugar  (36 tsp)
* 1 cup butter (48 tsp)
* 1 teaspoon vanilla (1 tsp)
* 1 egg
* 2 1/4 cups flour (108 tsp)
* 1 teaspoon baking soda (1 tsp)
* 1/2 teaspoon salt (0.5 tsp)
* 1 package (12 ounces) chocolate chips (2 cups) (96 tsp)

In [48]:
target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]
# sugar, brown sugar, butter, vanilla, egg,
# flour, baking soda, salt, choco chips



An example starting state for a member of our population might look like: $state = [30, 30, 40, 1, 0, 100, 0.5, 0.5, 100]$

### (3a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [74]:
def recipe_success(state):
    fitness = 1
    for i in range(0,len(state)):
        if state[i] != target[i]:
            diff = np.absolute(target[i] - state[i])
            fitness += diff
    return 1/fitness
            
recipe_success(state)

0.028985507246376812

### (3b) 

Using our in class notebook "CSCI3202_GeneticAlgorithm.ipynb" as your guide, write a genetic algorithm that starts with a population of 10 randomly generated "recipes/states/members" and uses the objective function you wrote in **(3a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from numbers between 0 and 108 (half step intervals might be helpful since the recipe requires 1/2 tsp salt)
- Allow for mutations in your population with an overall probability of mutation set to p = 0.1
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 200 iterations ("generations"). Do you hit your target?

In [150]:
# Your code here.
class problem:
    
    def __init__(self, initial_population, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)
        '''
        self.population = initial_population
        self.initial_population = initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.n_pop = len(initial_population)
        self.n_dna = len(initial_population[0])
        self.fitness_goal = fitness_goal

    def fitness(self):
        '''
        calculate each population member's probability of being selected for
        reproduction based on performance on objective function
        '''
        performance = []
        for k in range(self.n_pop):
            performance.append(self.objective_function(self.population[k]))
        total = sum(performance)
        p_reproduce = [perf/sum(performance) for perf in performance]
        return p_reproduce
        
    def reproduce(self, parent1, parent2):
        # last DNA snippet from parent1:
        split = np.random.randint(low=1, high=self.n_dna)
        child = parent1[:split] + parent2[split:]
        return child

    def mutate(self, child, change):
        # which gene to mutate?
        gene = np.random.randint(low=0, high=self.n_dna)
        
        if child[gene] > target[gene]:
            child[gene] -= np.random.randint(1,change)/2
        elif child[gene] < target[gene]:
            child[gene] += np.random.randint(1,change)/2
        return child



In [151]:
def genetic_algorithm(problem, n_iter, change):
    
    for t in range(n_iter):
        
        new_generation = []
        
        for k in range(problem.n_pop):
            
            # select for reproduction
            p_reproduce = problem.fitness()
            ind_parents = np.random.choice(range(0,problem.n_pop), size=2, p=p_reproduce, replace=False)
            parent1, parent2 = problem.population[ind_parents[0]], problem.population[ind_parents[1]]
            
            # reproduce
            child = problem.reproduce(parent1, parent2)
            
            # mutate
            l_mutate = np.random.choice([True, False], p=[problem.p_mutate, 1-problem.p_mutate])
            if l_mutate:
                child = problem.mutate(child, change)
            
            # add to new generation
            new_generation.append(child)
        
        # set problem.population = new generation
        problem.population = new_generation
        
        # exit criterion check
        performance = [problem.objective_function(member) for member in problem.population]
        
        best_member = max(zip(performance, problem.population))
        
        if best_member[0] >= problem.fitness_goal:
            return best_member, counter
        
        counter = t
        
    

    print('reached n_iter')

    return False

In [152]:
target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]
# sugar, brown sugar, butter, vanilla, egg,
# flour, baking soda, salt, choco chips

initial_population = []
for i in range(0,10):
    temp_arr = []
    for i in range(0,9):
        temp_arr.append((np.random.randint(0,216))/2)
    initial_population.append(temp_arr)


In [156]:
# THIS CODE IS TO CHECK THE NUMBER OF ITERATIONS IT TAKES WITH A SPECIFIC MUTATION
genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=1, 
                          mutation_probability=0.1, 
                          objective_function=recipe_success)
out, counter = genetic_algorithm(genetic_problem, 10000, 28)
print(out, counter)

(1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1867


In [155]:
# THIS CODE IS TO CHECK THE NUMBER OF ITERATIONS IT TAKES WITH MULTIPLE MUTATION
for i in range(10,40):
    genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=1, 
                          mutation_probability=0.1, 
                          objective_function=recipe_success)
    out, counter = genetic_algorithm(genetic_problem, 10000, i)
    print(i, out, counter)


10 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1799
11 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2018
12 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1538
13 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2556
14 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1555
15 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2050
16 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1123
17 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1123
18 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1668
19 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1217
20 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1708
21 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1842
22 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2190
23 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2041
24 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 

In [157]:
# THIS CODE IS TO CHECK THE AVERAGE NUMBER OF ITERATIONS
avg = 0
for i in range(0,10):
    genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=1, 
                          mutation_probability=0.1, 
                          objective_function=recipe_success)
    out, counter = genetic_algorithm(genetic_problem, 10000, 28)
    avg += counter
    print(i, out, counter)
    
print(avg/10)
    


0 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1093
1 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1468
2 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1948
3 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2011
4 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1462
5 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1128
6 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2101
7 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1521
8 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 1266
9 (1.0, [36.0, 36.0, 48.0, 1.0, 1.0, 108.0, 1.0, 0.5, 96.0]) 2270
1626.8


I have some extra code here as I am trying to find the best mutation integers so that it minimmizes the number of mutations it takes to reach the goal and found the with my code, the integer change in mutation is around 26-29 with a sweet spot at 27 or 28.

I definitely cannot reach my goal within 200 iterations.

With a mutation rate of randint(0,28)/2, I reach my target of all ingredients in ~1626 iterations.

### (3c)

Report the following:
- How many generations did it take to hit the goal?
- If you change the initial population size to 100, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation to 0.2, does that affect the number of generations it takes to achieve the goal recipe?

Now just answer whether you have minimized or maximized the maximize function?
In class notebook will be maximize.

How many ingredients did you match? How many iterations did it take? 2-6 in 600 iterations. 

Specify the initial population: [36, 0, 0, 0...] and [0, 36, 0, 0, ...]



- I maximized the maximize function.
- How many ingredients did you match? 
    - I was able to match all of the ingredients in ~1626 iterations.

- Specify the initial population: [36, 0, 0, 0...] and [0, 36, 0, 0, ...]
    - The initial population is a random list of random integers between 0 and 108 as the key components section specifies.